In [1]:
1+1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1564,application_1645475281079_151144,pyspark,idle,Link,Link,dksadykova,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [8]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 1. Let's download all our csv files:

In [3]:
path = '/user/dksadykova/hse_spark_hw_sonya/*.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
dataset = sc.textFile(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 2. Check that everything has been downloaded correctly:

In [7]:
dataset.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1010

In [ ]:
dataset.take(1)

#### As we can see, our dataset is not empty and the first string is shown as expected.

#### 3. Let's search for the top most popular words in our dataset:

In [36]:
top_df = dataset.flatMap(lambda x: x.split(' ')).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)\
.map(lambda x: (x[1], x[0])).sortByKey(False).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
for i in range(10):    print(top_df[i][1],top_df[i][0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The 199
John 112
the 95
of 88
Michael 83
James 71
David 63
Robert 50
Peter 49
Richard 42

#### As we can see, the most popular word is an article - 'the' - as well as some directors name (because dataset is dedicated to movies)

#### 4. Before we make any further calculations, we need to convert our dataset into dataframe, so it would be easier to analyze:

In [38]:
schema_s = 'line'
fields = [StructField(field_name, StringType(), True) for field_name in schema_s.split()]
schema = StructType(fields)
dataset_count = dataset.take(dataset.count())
dataset_mapped = sc.parallelize(dataset_count)
dataset_mapped = dataset_mapped.map(lambda x:[x])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
dataframe = spark.createDataFrame(dataset_mapped, schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4. After we have created the dataframe, let's calculate how many words it consists of:

In [40]:
dataframe = dataframe.withColumn('word_count', f.size(f.split(f.col('line'), ' ')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
dataframe.select(f.sum('word_count')).collect() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(sum(word_count)=16395)]

##### This means that there are more than 15k words in this file, which makes it suitable for relevant analysis